In [1]:
import numpy as np

In [2]:
def evalf(x):
  assert type(x) is np.ndarray and len(x)==3
  return (((x[0]-2)**2)/8 + ((x[1]-4)**2)/64 + ((x[2]-8)**2)/512)

In [3]:
def evalg(x):
  assert type(x) is np.ndarray and len(x)==3
  return np.array([(x[0]-2)/4 ,(x[1]-4)/32 ,(x[2]-8)/256])

In [4]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_exact(gradf, A): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  assert type(A) is np.ndarray and A.shape[0] == 3 and  A.shape[1] == 3 #allow only a 2x2 array
   
  #Complete the code to compute step length
  step_length=(np.dot(gradf.T,gradf)) / (np.matmul(np.matmul(gradf,2*A),gradf)) 
  
  return step_length

In [5]:
#Complete the module to compute the steplength by using the backtracking line search
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 3 
  assert type(gradf) is np.ndarray and len(gradf) == 3 
  
  alpha = alpha_start
  p=rho
  y=gamma
  #implement the backtracking line search
  while evalf(x+alpha*(-gradf)) > evalf(x)-y*alpha*np.dot((gradf.T),gradf):
    alpha=p*alpha


  #print('final step length:',alpha)
  return alpha

In [6]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [7]:
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 3 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0, 0],[0, 1/64, 0],[0, 0, 1/512]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,k

$\huge{2.}$ \
The code is implemented below:

In [8]:
my_start_x=np.array([1,1,1])
my_tol=10**(-5)



x_opt,iterations = find_minimizer(my_start_x, my_tol, CONSTANT_STEP_LENGTH)
print('Minimizer for constant step length =',x_opt)
print('Minimum function value =',evalf(x_opt))

#check what happens when you call find_minimzer using backtracking line search
x_opt_bls,iterations = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('Minimizer for constant backtracking line search =',x_opt_bls)
print('Minimum function value =',evalf(x_opt_bls))

Minimizer for constant step length = [2.         4.         7.99744099]
Minimum function value = 1.2790080180472962e-08
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer for constant backtracking line search = [2.         4.         7.99744063]
Minimum function value = 1.2793697352715871e-08


$\huge{3.}$ \
The code is implemented below:

In [9]:
my_start_x=np.array([1/64,1/8,1])
my_tol=10**(-10)
x_opt,itr = find_minimizer(my_start_x, my_tol, EXACT_LINE_SEARCH)
print('Minimizer for exact step length =',x_opt)
print('Minimum function value =',evalf(x_opt))
print('Number of iterations =',itr)

Minimizer for exact step length = [2.         4.         7.99999998]
Minimum function value = 9.150071377581033e-19
Number of iterations = 269


In [10]:
my_start_x = np.array([1/64,1/8,1])
my_tol= 1e-10
x_opt_bls,itr = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print('Minimizer for constant backtracking line search =',x_opt_bls)
print('Minimum function value =',evalf(x_opt_bls))
print('Number of iterations =',itr)

Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer for constant backtracking line search = [2.         4.         7.99999997]
Minimum function value = 1.2748574165464873e-18
Number of iterations = 4964


**Observations:** We can see that the number of iterations to terminate the Backtracking search (4964) is higher than the number of iterations to terminate Exact line search(269). From this, we can conclude that in this case, the exact line search turns out to be the faster algorithm.

$\huge{4.}$ \
The code is implemented below:

In [11]:
def evalf(x):
  assert type(x) is np.ndarray and len(x)==4
  return (((x[0]-2)**2)/8 + ((x[1]-4)**2)/64 + ((x[2]-8)**2)/512+ ((x[3]-16)**2)/4096)

In [12]:
def evalg(x):
  assert type(x) is np.ndarray and len(x)==4
  return np.array([(x[0]-2)/4 ,(x[1]-4)/32 ,(x[2]-8)/256,(x[3]-16)/2048])

In [13]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_exact(gradf, A): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 4 
  assert type(A) is np.ndarray and A.shape[0] == 4 and  A.shape[1] == 4 #allow only a 2x2 array
   
  #Complete the code to compute step length
  step_length=(np.dot(gradf.T,gradf)) / (np.matmul(np.matmul(gradf,2*A),gradf)) 
  
  return step_length

In [14]:
#Complete the module to compute the steplength by using the backtracking line search
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 4
  assert type(gradf) is np.ndarray and len(gradf) == 4 
  
  alpha = alpha_start
  p=rho
  y=gamma
  #implement the backtracking line search
  while evalf(x+alpha*(-gradf)) > evalf(x)-y*alpha*np.dot((gradf.T),gradf):
    alpha=p*alpha


  #print('final step length:',alpha)
  return alpha

In [15]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [16]:
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1, 0],[0,1]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,k
def find_minimizer(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 4 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  # construct a suitable A matrix for the quadratic function 
  A = np.array([[1/8, 0, 0 ,0],[0,1/64, 0 ,0],[0,0,1/512,0],[0 ,0 ,0,1/4096 ]])
  x = start_x
  g_x = evalg(x)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
  
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
      #raise ValueError('EXACT LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
      #raise ValueError('BACKTRACKING LINE SEARCH NOT YET IMPLEMENTED')
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #implement the gradient descent steps here   
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x ,k

In [18]:
my_start_x=np.array([1/512,1/64,1/8,1])
my_tol=10**(-10)
x_opt,itr = find_minimizer(my_start_x, my_tol, EXACT_LINE_SEARCH)
print('Minimizer for constant step length =',x_opt)
print('Minimum function value =',evalf(x_opt))
print('Number of iterations =',itr)

Minimizer for constant step length = [ 2.          4.          8.         15.99999981]
Minimum function value = 8.8565993523583e-18
Number of iterations = 2013


In [20]:
my_start_x=np.array([1/512,1/64,1/8,1])
my_tol=10**(-10)
x_opt,itr = find_minimizer(my_start_x, my_tol, BACKTRACKING_LINE_SEARCH,1,0.5,0.5)
print('Minimizer for constant backtracking line search =',x_opt)
print('Minimum function value =',evalf(x_opt))
print('Number of iterations =',itr)

Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
Minimizer for constant backtracking line search = [ 2.         4.         8.        15.9999998]
Minimum function value = 1.0237544252113035e-17
Number of iterations = 37079


**Observations:** We observe that the number of iterations to terminate Backtracking search (37079) is higher than the number of iterations to terminate Exact line search (2013). By this, we can conclude that in this case, the exact line search is a faster algorithm compared to the backtracking line search.

$\huge{5.}$ \
\
$\large{N > 4}$ \

\
We saw above that the exact line search algorithm took 269 and 2013 iterations respectively to terminate for N=3 and 4. We can say based on this that as we increase the value of N by 1, the number of iterations increases by a good factor. Hence, the number of iterations to terminate for N>4 will be very high.

We saw above that the Backtracking line search algorithm took 4964 and 37079 iterations respectively to terminate for N=3 and 4. We can say based on this that as we increase the value of N by 1 the number of iteration increases by a good factor. Hence, the number of iterations to terminate for N>4 will be very high.

Further, the exact line search algorithm is taking less number of iterations than the Backtracking line search. Thus, the exact line search algorithm is more suitable for N>4.